In [1]:
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import cv2
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import NASNetLarge
from sklearn.decomposition import PCA
import joblib

2024-05-14 23:00:33.287345: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 23:00:33.470597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)

2.12.1


In [3]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
print(gpu_available, is_cuda_gpu_available, is_cuda_gpu_min_3)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True True True


2024-05-14 23:00:37.343623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 23:00:37.456072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 23:00:37.456132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 23:00:37.624338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 23:00:37.624457: I tensorflow/compile

In [4]:
# Path to the directory containing the images
data_dir = "Dataset"
dataset_paths = ["Dataset/correct_filters", "Dataset/defect_filters"]

In [5]:
# Load images and labels
images = []
labels = []

for i, dataset_path in enumerate(dataset_paths):
    for filename in os.listdir(dataset_path):
        img = cv2.imread(os.path.join(dataset_path, filename))
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_LANCZOS4)
        images.append(img)
        labels.append(i)

In [6]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [7]:
# Shuffle the data
images, labels = shuffle(images, labels, random_state=42)

In [8]:
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2222, stratify=y_train, random_state=42)

In [9]:
len(X_train)

3906

In [10]:
# Flatten the images
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_test.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [11]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_flat, y_train)

In [12]:
# Apply Tomek links
tomek = TomekLinks()
X_train_resampled, y_train_resampled = tomek.fit_resample(X_train_smote, y_train_smote)
# Use SMOTE data for training and original validation data for validation
X_val_smote, y_val_smote = X_val_flat, y_val

In [13]:
len(X_train)

3906

In [14]:
np.save('y_train.npy', y_train)
np.save('y_val.npy', y_val)
np.save('y_test.npy', y_test)

In [15]:
# # Check the number of samples in each class after resampling
# unique_train, counts_train = np.unique(y_train_resampled, return_counts=True)
# print(dict(zip(unique_train, counts_train)))

In [16]:
# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [17]:
len(X_train)

3906

In [18]:
# Reshape the data for PCA
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_val_reshaped = X_val.reshape(X_val.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

In [19]:
# Clear previous session
tf.keras.backend.clear_session()

In [20]:
len(X_train)

3906

In [21]:
# Apply PCA
pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train_reshaped)
X_val_pca = pca.transform(X_val_reshaped)
X_test_pca = pca.transform(X_test_reshaped)

: 

In [ ]:
# Save PCA model and transformed data
joblib.dump(pca, 'pca_model.pkl')
np.save('X_train_pca.npy', X_train_pca)
np.save('X_val_pca.npy', X_val_pca)
np.save('X_test_pca.npy', X_test_pca)

In [ ]:
# Number of components
num_components = pca.n_components_
print("Number of components:", num_components)

Number of components: 377


In [24]:
# # Print the shape of the components
# print("Shape of components:", sparse_pca.components_.shape)

In [25]:
# # Perform t-SNE
# tsne = TSNE(n_components=2, random_state=42)
# X_train_tsne = tsne.fit_transform(X_train_pca)

In [21]:

# # Create a DataFrame with t-SNE dimensions and labels
# df_tsne = pd.DataFrame(data = {
#     'Dim_1': X_train_tsne[:, 0],
#     'Dim_2': X_train_tsne[:, 1],
#     'label': y_train_resampled
# })

In [22]:
# # Plot the results
# plt.figure(figsize=(8,8))
# sns.scatterplot(
#     x="Dim_1", y="Dim_2",
#     hue="label",
#     palette=sns.color_palette("hsv", len(df_tsne['label'].unique())),
#     data=df_tsne,
#     legend="full",
#     alpha=0.3
# )
# plt.show()

In [23]:
# # Data Augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

In [24]:
# # Define the CNN model
# model = Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     layers.MaxPooling2D(2, 2),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#     layers.Flatten(),
#     layers.Dense(512, activation='relu'),
#     layers.Dropout(0.5),
#     layers.Dense(1, activation='sigmoid')
# ])

In [25]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
# batch = 16
# img_height = 224
# img_width = 224
# Channels = 3

In [27]:
# # Load the dataset
# train_ds = tf.keras.utils.image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="training",
#     seed=42,
#     image_size=(img_height, img_width),
#     batch_size=batch)

In [28]:
# val_ds = tf.keras.utils.image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="validation",
#   seed=42,
#   image_size=(img_height, img_width),
#   batch_size=batch)

In [29]:
# # This is to improve the training time I guess
# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [30]:
# Define and train the autoencoder
autoencoder = tf.keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(4, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(2, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(2, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(4, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

autoencoder.compile(optimizer='adam', loss='mse')

2024-05-14 20:31:16.048011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 20:31:16.048261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 20:31:16.048322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 20:31:16.048854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 20:31:16.048934: I tensorflow/compile

In [31]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 8)       224       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 4)       292       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 4)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 2)         74        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 2)        0

In [32]:
autoencoder.fit(X_train, X_train, epochs=10, batch_size=1, validation_data=(X_val, X_val))

Epoch 1/10


2024-05-14 20:31:24.387346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8907
2024-05-14 20:31:25.469244: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f19e1907b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-14 20:31:25.469297: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-05-14 20:31:25.510184: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-14 20:31:25.791061: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1458/1458 [==============================] - 15s 6ms/step - loss: 0.0115 - val_loss: 0.0024
Epoch 2/10
1458/1458 [==============================] - 8s 6ms/step - loss: 0.0049 - val_loss: 0.0022
Epoch 3/10
1458/1458 [==============================] - 8s 6ms/step - loss: 0.0043 - val_loss: 0.0018
Epoch 4/10
1458/1458 [==============================] - 8s 6ms/step - loss: 0.0040 - val_loss: 0.0016
Epoch 5/10
1458/1458 [==============================] - 8s 5ms/step - loss: 0.0037 - val_loss: 0.0016
Epoch 6/10
1458/1458 [==============================] - 8s 6ms/step - loss: 0.0037 - val_loss: 0.0015
Epoch 7/10
1458/1458 [==============================] - 8s 5ms/step - loss: 0.0035 - val_loss: 0.0015
Epoch 8/10
1458/1458 [==============================] - 8s 5ms/step - loss: 0.0035 - val_loss: 0.0015
Epoch 9/10
1458/1458 [==============================] - 8s 5ms/step - loss: 0.0034 - val_loss: 0.0014
Epoch 10/10
1458/1458 [==============================] - 8s 5ms/step - loss: 0.0034 - val_lo

In [33]:
tf.keras.backend.clear_session()

In [35]:
# Use the encoder part of the autoencoder to compress the image data
encoder = tf.keras.Sequential(autoencoder.layers[:15])
encoder.compile()

# Set device to CPU
with tf.device('/CPU:0'):
    X_train_encoded = encoder.predict(X_train)
    X_val_encoded = encoder.predict(X_val)
    X_test_encoded = encoder.predict(X_test)

7/7 [==============================] - 0s 32ms/step


In [36]:
# num_classes = 2

# model = tf.keras.Sequential([
#     layers.Rescaling(1./255),
#     data_augmentation,
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(128, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(256, 3, padding='same', activation='relu'),  # New convolutional layer
#     layers.MaxPooling2D(),
#     layers.Conv2D(1024, 3, padding='same', activation='relu'),  # New convolutional layer
#     layers.MaxPooling2D(),
#     layers.Conv2D(2048, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(4096, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dropout(0.5),
#     layers.Dense(256, activation='relu'),  # Increased dense layer size
#     layers.Dense(num_classes, activation='softmax')
# ])

# base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
#                         include_top=False,
#                         weights='imagenet')

# base_model.trainable = False

# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# # Build the model
# model = tf.keras.Sequential([
#     layers.Rescaling(1./255),
#     data_augmentation,
#     base_model,
#     global_average_layer,
#     layers.Dropout(0.5),
#     prediction_layer
# ])

# base_model = EfficientNetB0(input_shape=(img_height, img_width, 3),
#                             include_top=False,
#                             weights='imagenet')

# base_model.trainable = False

# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# dense_layer1 = tf.keras.layers.Dense(512, activation='relu')
# dropout_layer1 = tf.keras.layers.Dropout(0.5)
# dense_layer2 = tf.keras.layers.Dense(256, activation='relu')
# dropout_layer2 = tf.keras.layers.Dropout(0.5)

# prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# # Build the model
# model = tf.keras.Sequential([
#     layers.Rescaling(1./255),
#     data_augmentation,
#     base_model,
#     global_average_layer,
#     dense_layer1,
#     dropout_layer1,
#     dense_layer2,
#     dropout_layer2,
#     prediction_layer
# ])

# base_model = DenseNet201(input_shape=(img_height, img_width, 3),
#                          include_top=False,
#                          weights='imagenet')

# base_model.trainable = False

# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# dense_layer1 = tf.keras.layers.Dense(1024, activation='relu')
# dropout_layer1 = tf.keras.layers.Dropout(0.5)
# dense_layer2 = tf.keras.layers.Dense(512, activation='relu')
# dropout_layer2 = tf.keras.layers.Dropout(0.5)

# prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# # Build the model
# model = tf.keras.Sequential([
#     layers.Rescaling(1./255),
#     data_augmentation,
#     base_model,
#     global_average_layer,
#     dense_layer1,
#     dropout_layer1,
#     dense_layer2,
#     dropout_layer2,
#     prediction_layer
# ])

# This model works with layers.RandomFlip("horizontal_and_vertical") only. That's how complex it needs to be.
base_model = NASNetLarge(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

dense_layer1 = tf.keras.layers.Dense(4096, activation='relu')
dropout_layer1 = tf.keras.layers.Dropout(0.5)
dense_layer2 = tf.keras.layers.Dense(2048, activation='relu')
dropout_layer2 = tf.keras.layers.Dropout(0.5)
dense_layer3 = tf.keras.layers.Dense(1024, activation='relu')
dropout_layer3 = tf.keras.layers.Dropout(0.5)

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# Build the model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])



# base_model = NASNetLarge(input_shape=(224, 224, 3),
#                          include_top=False,
#                          weights='imagenet')

# # Fine-tuning
# base_model.trainable = False
# # fine_tune_at = 900  # You can adjust this
# # for layer in base_model.layers[:fine_tune_at]:
# #     layer.trainable = False

# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# dense_layer1 = tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
# dropout_layer1 = tf.keras.layers.Dropout(0.5)
# dense_layer2 = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
# dropout_layer2 = tf.keras.layers.Dropout(0.5)
# dense_layer3 = tf.keras.layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
# dropout_layer3 = tf.keras.layers.Dropout(0.5)

# prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# # Build the model
# model = tf.keras.Sequential([
#     base_model,
#     global_average_layer,
#     dense_layer1,
#     dropout_layer1,
#     dense_layer2,
#     dropout_layer2,
#     dense_layer3,
#     dropout_layer3,
#     prediction_layer
# ])

# base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
#                                             include_top=False,
#                                             weights='imagenet')

# # Fine-tuning
# base_model.trainable = False

# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# # Build the model
# model = tf.keras.Sequential([
#     base_model,
#     global_average_layer,
#     prediction_layer
# ])

In [37]:
# print("Number of layers in the base model: ", len(base_model.layers))

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
# Define callbacks
checkpoint_filepath = 'model_checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


In [40]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=25,
    min_delta=0.001,
    restore_best_weights=True)

In [41]:
# Load the weights from the checkpoint
# model.load_weights(checkpoint_filepath)

In [42]:
tf.keras.backend.clear_session()

In [43]:
# Fit the model
history = model.fit(
    X_train_encoded,
    y_train_resampled,
    validation_data=(X_val_encoded, y_val),
    epochs=50,
    steps_per_epoch=len(X_train),
    batch_size=2,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

Epoch 1/50


2024-05-14 20:34:10.625369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1458/1458 [==============================] - 98s 60ms/step - loss: 0.5706 - accuracy: 0.6979 - val_loss: 0.5587 - val_accuracy: 0.6919
Epoch 2/50
1458/1458 [==============================] - 83s 57ms/step - loss: 0.4639 - accuracy: 0.7891 - val_loss: 0.5032 - val_accuracy: 0.7626
Epoch 3/50
1458/1458 [==============================] - 83s 57ms/step - loss: 0.4227 - accuracy: 0.8042 - val_loss: 0.4923 - val_accuracy: 0.7576
Epoch 4/50
1458/1458 [==============================] - 79s 54ms/step - loss: 0.3793 - accuracy: 0.8275 - val_loss: 0.5213 - val_accuracy: 0.7424
Epoch 5/50
1458/1458 [==============================] - 80s 55ms/step - loss: 0.3500 - accuracy: 0.8364 - val_loss: 0.4619 - val_accuracy: 0.7727
Epoch 6/50
1458/1458 [==============================] - 99s 68ms/step - loss: 0.3238 - accuracy: 0.8536 - val_loss: 0.4492 - val_accuracy: 0.7828
Epoch 7/50
1458/1458 [==============================] - 88s 60ms/step - loss: 0.3007 - accuracy: 0.8765 - val_loss: 0.4437 - val_accura

KeyboardInterrupt: 